# Eager Few Shot Object Detection Colab

Welcome to the Eager Few Shot Object Detection Colab --- in this colab we demonstrate fine tuning of a (TF2 friendly) RetinaNet architecture on very few examples of a novel class after initializing from a pre-trained COCO checkpoint.
Training runs in eager mode.

To run this colab, you will need to connect to a borg runtime (we recommend
Tensorflow GPU with Python 3).

Estimated time to run through this colab: < 5 minutes.

## Imports

In [ ]:
!pip install -U --pre tensorflow=="2.2.0"

In [ ]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
# Test the Object Detection API installation
%%bash
cd models/research
python object_detection/builders/model_builder_tf2_test.py

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import uuid
import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

from IPython.display import display, Javascript
from google.colab import output
from google.colab.output import eval_js
from base64 import b64decode, b64encode
from typing import List, Dict, Union
import operator
import json

%matplotlib inline

# Utilities

In [ ]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: a file path (this can be local or on colossus)

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def plot_detections(image_np,
                    boxes,
                    classes,
                    scores,
                    category_index, 
                    figsize=(12, 16),
                    image_name=None):
  """Wrapper function to visualize detections.

  Args:
    image_np: uint8 numpy array with shape (img_height, img_width, 3)
    boxes: a numpy array of shape [N, 4]
    classes: a numpy array of shape [N]. Note that class indices are 1-based,
      and match the keys in the label map.
    scores: a numpy array of shape [N] or None.  If scores=None, then
      this function assumes that the boxes to be plotted are groundtruth
      boxes and plot all boxes as black with no classes or scores.
    category_index: a dict containing category dictionaries (each holding
      category index `id` and category name `name`) keyed by category indices.
  """
  image_np_with_annotations = image_np.copy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_annotations,
      boxes,
      classes,
      scores,
      category_index,
      use_normalized_coordinates=True,
      min_score_thresh=0.8)
  if (image_name):
    plt.imsave(image_name, image_np_with_annotations)
  else:
    plt.imshow(image_np_with_annotations)


def image_from_numpy(image):
    """
    Open an image at the specified path and encode it in Base64.

    Parameters
    ----------
    image: np.ndarray
        Image represented as a numpy array

    Returns
    -------
    str
        Encoded Base64 representation of the image
    """

    with io.BytesIO() as output:
        Image.fromarray(image).save(output, format="JPEG")
        data = output.getvalue()
    data = str(b64encode(data))[2:-1]
    return data

def draw_bbox(image_urls, callbackId):
    """
    Open the bounding box UI and send the results to a callback function.

    Parameters
    ----------
    image_urls: list[str | np.ndarray]
        List of locations from where to load the images from. If a np.ndarray is
        given, the array is interpretted as an image and sent to the frontend. If
        a str is given, the string is interpreted as a path and is read as a
        np.ndarray before being sent to the frontend.

    callbackId: str
        The ID for the callback function to send the bounding box results to
        when the user hits submit.
    """
    js = Javascript('''
                async function load_image(imgs, callbackId) {
                    //init organizational elements
                    const div = document.createElement('div');
                    var image_cont = document.createElement('div');
                    var errorlog = document.createElement('div');
                    var crosshair_h = document.createElement('div');
                    crosshair_h.style.position = "absolute";
                    crosshair_h.style.backgroundColor = "transparent";
                    crosshair_h.style.width = "100%";
                    crosshair_h.style.height = "0px";
                    crosshair_h.style.zIndex = 9998;
                    crosshair_h.style.borderStyle = "dotted";
                    crosshair_h.style.borderWidth = "2px";
                    crosshair_h.style.borderColor = "rgba(255, 0, 0, 0.75)";
                    crosshair_h.style.cursor = "crosshair";
                    var crosshair_v = document.createElement('div');
                    crosshair_v.style.position = "absolute";
                    crosshair_v.style.backgroundColor = "transparent";
                    crosshair_v.style.width = "0px";
                    crosshair_v.style.height = "100%";
                    crosshair_v.style.zIndex = 9999;
                    crosshair_v.style.top = "0px";
                    crosshair_v.style.borderStyle = "dotted";
                    crosshair_v.style.borderWidth = "2px";
                    crosshair_v.style.borderColor = "rgba(255, 0, 0, 0.75)";
                    crosshair_v.style.cursor = "crosshair";
                    crosshair_v.style.marginTop = "23px";
                    var brdiv = document.createElement('br');


                    //init control elements
                    var next = document.createElement('button');
                    var prev = document.createElement('button');
                    var submit = document.createElement('button');
                    var deleteButton = document.createElement('button');
                    var deleteAllbutton = document.createElement('button');

                    //init image containers
                    var image = new Image();
                    var canvas_img = document.createElement('canvas');
                    var ctx = canvas_img.getContext("2d");
                    canvas_img.style.cursor = "crosshair";
                    canvas_img.setAttribute('draggable', false);
                    crosshair_v.setAttribute('draggable', false);
                    crosshair_h.setAttribute('draggable', false);

                    // bounding box containers
                    const height = 600
                    var allBoundingBoxes = [];
                    var curr_image = 0
                    var im_height = 0;
                    var im_width = 0;

                    //initialize bounding boxes
                    for (var i = 0; i < imgs.length; i++) {
                      allBoundingBoxes[i] = [];
                    }
                    
                    //initialize image view
                    errorlog.id = 'errorlog';
                    image.style.display = 'block';
                    image.setAttribute('draggable', false);

                    //load the first image
                    img = imgs[curr_image];
                    image.src = "data:image/png;base64," + img;
                    image.onload = function() {
                        // normalize display height and canvas
                        image.height = height;
                        image_cont.height = canvas_img.height = image.height;
                        image_cont.width = canvas_img.width = image.naturalWidth;
                        crosshair_v.style.height = image_cont.height + "px";
                        crosshair_h.style.width = image_cont.width + "px";

                        // draw the new image
                        ctx.drawImage(image, 0, 0, image.naturalWidth, image.naturalHeight, 0, 0,  canvas_img.width,  canvas_img.height);

                    };

                    // move to next image in array
                    next.textContent = "next image";
                    next.onclick = function(){
                        if (curr_image < imgs.length - 1){
                            // clear canvas and load new image
                            curr_image += 1;
                            errorlog.innerHTML = "";
                        }
                        else{
                            errorlog.innerHTML = "All images completed!!";
                        }
                        resetcanvas();
                    }

                    //move forward through list of images
                    prev.textContent = "prev image"
                    prev.onclick = function(){
                        if (curr_image > 0){
                            // clear canvas and load new image
                            curr_image -= 1;
                            errorlog.innerHTML = "";
                        }
                        else{
                            errorlog.innerHTML = "at the beginning";
                        }
                        resetcanvas();
                    }
                    
                    // on delete, deletes the last bounding box
                    deleteButton.textContent = "undo bbox";
                    deleteButton.onclick = function(){
                      boundingBoxes.pop();
                      ctx.clearRect(0, 0, canvas_img.width, canvas_img.height);
                      image.src = "data:image/png;base64," + img;
                      image.onload = function() {
                          ctx.drawImage(image, 0, 0, image.naturalWidth, image.naturalHeight, 0, 0,  canvas_img.width,  canvas_img.height);
                          boundingBoxes.map(r => {drawRect(r)});
                      };
                    }
                    
                    // on all delete, deletes all of the bounding box
                    deleteAllbutton.textContent = "delete all"
                    deleteAllbutton.onclick = function(){
                      boundingBoxes = [];
                      ctx.clearRect(0, 0, canvas_img.width, canvas_img.height);
                      image.src = "data:image/png;base64," + img;
                      image.onload = function() {
                          ctx.drawImage(image, 0, 0, image.naturalWidth, image.naturalHeight, 0, 0,  canvas_img.width,  canvas_img.height);
                          //boundingBoxes.map(r => {drawRect(r)});
                      };
                    }

                    // on submit, send the boxes to display
                    submit.textContent = "submit";
                    submit.onclick = function(){
                      errorlog.innerHTML = "";

                      // send box data to callback fucntion
                      google.colab.kernel.invokeFunction(callbackId, [allBoundingBoxes], {});
                    }

                  // init template for annotations
                  const annotation = {
                        x: 0,
                        y: 0,
                        w: 0,
                        h: 0,
                  };

                  // the array of all rectangles
                  let boundingBoxes = allBoundingBoxes[curr_image];

                  // the actual rectangle, the one that is being drawn
                  let o = {};

                  // a variable to store the mouse position
                  let m = {},

                  // a variable to store the point where you begin to draw the
                  // rectangle
                  start = {};

                  // a boolean variable to store the drawing state
                  let isDrawing = false;
                  var elem = null;

                  function handleMouseDown(e) {
                    // on mouse click set change the cursor and start tracking the mouse position
                    start = oMousePos(canvas_img, e);

                    // configure is drawing to true
                    isDrawing = true; 
                  }

                  function handleMouseMove(e) {
                      // move crosshairs, but only within the bounds of the canvas
                      if (document.elementsFromPoint(e.pageX, e.pageY).includes(canvas_img)) {
                        crosshair_h.style.top = e.pageY + "px";
                        crosshair_v.style.left = e.pageX + "px";
                      }

                      // move the bounding box
                      if(isDrawing){
                        m = oMousePos(canvas_img, e);
                        draw();
                      }
                  }

                  function handleMouseUp(e) {
                      if (isDrawing) {
                          // on mouse release, push a bounding box to array and draw all boxes
                          isDrawing = false;

                          const box = Object.create(annotation);

                          // calculate the position of the rectangle
                          if (o.w > 0){
                            box.x = o.x;
                          }
                          else{
                            box.x = o.x + o.w; 
                          }
                          if (o.h > 0){
                            box.y = o.y;
                          }
                          else{
                            box.y = o.y + o.h; 
                          }
                          box.w = Math.abs(o.w); 
                          box.h = Math.abs(o.h);

                          // add the bounding box to the image
                          boundingBoxes.push(box);
                          draw();
                      }
                  }

                  function draw() {  
                      o.x = (start.x)/image.width;  // start position of x
                      o.y = (start.y)/image.height;  // start position of y
                      o.w = (m.x - start.x)/image.width;  // width
                      o.h = (m.y - start.y)/image.height;  // height

                      ctx.clearRect(0, 0, canvas_img.width, canvas_img.height);
                      ctx.drawImage(image, 0, 0, image.naturalWidth, image.naturalHeight, 0, 0,  canvas_img.width,  canvas_img.height);
                      // draw all the rectangles saved in the rectsRy
                      boundingBoxes.map(r => {drawRect(r)});
                      // draw the actual rectangle
                      drawRect(o);  
                  }

                  // add the handlers needed for dragging
                  crosshair_h.addEventListener("mousedown", handleMouseDown);
                  crosshair_v.addEventListener("mousedown", handleMouseDown);
                  document.addEventListener("mousemove", handleMouseMove);
                  document.addEventListener("mouseup", handleMouseUp);


                  function resetcanvas(){
                      // clear canvas
                      ctx.clearRect(0, 0, canvas_img.width, canvas_img.height);
                      img = imgs[curr_image]
                      image.src = "data:image/png;base64," + img;

                      // onload init new canvas and display image 
                      image.onload = function() {
                          // normalize display height and canvas
                          image.height = height;
                          image_cont.height = canvas_img.height = image.height;
                          image_cont.width = canvas_img.width = image.naturalWidth;
                          crosshair_v.style.height = image_cont.height + "px";
                          crosshair_h.style.width = image_cont.width + "px";

                          // draw the new image
                          ctx.drawImage(image, 0, 0, image.naturalWidth, image.naturalHeight, 0, 0,  canvas_img.width,  canvas_img.height);

                          // draw bounding boxes
                          boundingBoxes = allBoundingBoxes[curr_image];
                          boundingBoxes.map(r => {drawRect(r)});
                      };
                  }

                  function drawRect(o){
                      // draw a predefined rectangle 
                      ctx.strokeStyle = "red";
                      ctx.lineWidth = 2;
                      ctx.beginPath(o);
                      ctx.rect(o.x * image.width, o.y * image.height, o.w * image.width, o.h * image.height);
                      ctx.stroke();
                  }

                  // Function to detect the mouse position
                  function oMousePos(canvas_img, evt) {
                    let ClientRect = canvas_img.getBoundingClientRect();
                      return { 
                        x: evt.clientX - ClientRect.left,
                        y: evt.clientY - ClientRect.top
                      };
                  }


                  //configure colab output display
                  google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

                  //build the html document that will be seen in output
                  div.appendChild(document.createElement('br'))
                  div.appendChild(image_cont)
                  image_cont.appendChild(canvas_img)
                  image_cont.appendChild(crosshair_h)
                  image_cont.appendChild(crosshair_v)
                  div.appendChild(document.createElement('br'))
                  div.appendChild(errorlog)
                  div.appendChild(prev)
                  div.appendChild(next)
                  div.appendChild(deleteButton)
                  div.appendChild(deleteAllbutton)
                  div.appendChild(document.createElement('br'))
                  div.appendChild(brdiv)
                  div.appendChild(submit)
                  document.querySelector("#output-area").appendChild(div);
                  return
              }''')
    
    #load the images as a byte array
    bytearrays = []
    for image in image_urls:
        if isinstance(image, str):
            bytearrays.append(image_from_path(image))
        elif isinstance(image, np.ndarray):
            bytearrays.append(image_from_numpy(image))
        else:
            raise TypeError(f"Image has unsupported type {type(image)}. Only str and np.ndarray are supported.")

    #format arrays for input
    image_data = json.dumps(bytearrays)
    del bytearrays

    #call java script function pass string byte array(image_data) as input
    display(js)
    eval_js(f"load_image({image_data}, '{callbackId}')")
    return

def annotate(imgs: List[Union[str, np.ndarray]], box_storage_pointer: List[np.ndarray], callbackId: str = None):
  """
  Open the bounding box UI and prompt the user for input.

  Parameters
  ----------
  imgs: list[str | np.ndarray]
      List of locations from where to load the images from. If a np.ndarray is
      given, the array is interpretted as an image and sent to the frontend. If
      a str is given, the string is interpreted as a path and is read as a
      np.ndarray before being sent to the frontend.

  box_storage_pointer: list[np.ndarray]
      Destination list for bounding box arrays. Each array in this list
      corresponds to one of the images given in imgs. The array is a
      N x 4 array where N is the number of bounding boxes given by the user
      for that particular image. If there are no bounding boxes for an image,
      None is used instead of an empty array.

  callbackId: str, optional
      The ID for the callback function that communicates between the fontend
      and the backend. If no ID is given, a random UUID string is used instead.
  """

  # Set a random ID for the callback function
  if callbackId is None:
    callbackId = str(uuid.uuid1()).replace('-', '')

  def dictToList(input):
    '''
    This function converts the dictionary from the frontend (if the format
    {x, y, w, h} as shown in callbackFunction) into a list
    ([y_min, x_min, y_max, x_max])
    '''

    return (input['y'], input['x'], input['y'] + input['h'], input['x'] + input['w'])

  def callbackFunction(annotations: List[List[Dict[str, float]]]):
    """
    This is the call back function to capture the data from the frontend and
    convert the data into a numpy array.

    Parameters
    ----------
    annotations: list[list[dict[str, float]]]
        The input of the call back function is a list of list of objects
        corresponding to the annotations. The format of annotations is shown
        below

        [
          // stuff for image 1
          [
            // stuff for rect 1
            {x, y, w, h},
            // stuff for rect 2
            {x, y, w, h},
            ...
          ],
          // stuff for image 2
          [
            // stuff for rect 1
            {x, y, w, h},
            // stuff for rect 2
            {x, y, w, h},
            ...
          ],
          ...
        ]
    """

    # reset the boxes list
    nonlocal box_storage_pointer
    boxes: List[np.ndarray] = box_storage_pointer
    boxes.clear()

    # load the new annotations into the boxes list
    for annotationsPerImg in annotations:
      rectanglesAsArrays = [np.clip(dictToList(annotation), 0, 1) for annotation in annotationsPerImg]
      if rectanglesAsArrays:
        boxes.append(np.stack(rectanglesAsArrays))
      else:
        boxes.append(None)

    # output the annotations to the errorlog
    with output.redirect_to_element('#errorlog'):
      display("--boxes array populated--")
      
  output.register_callback(callbackId, callbackFunction)
  draw_bbox(imgs, callbackId)

# Rubber Ducky data

We will start with some toy (literally) data consisting of 5 images of a rubber
ducky.  Note that the [coco](https://cocodataset.org/#explore) dataset contains a number of animals, but notably, it does *not* contain rubber duckies (or even ducks for that matter), so this is a novel class.

In [ ]:
# Load images and visualize
train_image_dir = 'models/research/object_detection/test_images/ducky/train/'
train_images_np = []
for i in range(1, 6):
  image_path = os.path.join(train_image_dir, 'robertducky' + str(i) + '.jpg')
  train_images_np.append(load_image_into_numpy_array(image_path))

plt.rcParams['axes.grid'] = False
plt.rcParams['xtick.labelsize'] = False
plt.rcParams['ytick.labelsize'] = False
plt.rcParams['xtick.top'] = False
plt.rcParams['xtick.bottom'] = False
plt.rcParams['ytick.left'] = False
plt.rcParams['ytick.right'] = False
plt.rcParams['figure.figsize'] = [14, 7]

for idx, train_image_np in enumerate(train_images_np):
  plt.subplot(2,3, idx+1)
  plt.imshow(train_image_np)
plt.show()

# Annotate images with bounding boxes

In this cell you will annotate the rubber duckies --- draw a box around the rubber ducky in each image; click `next image` to go to the next image and `submit` when there are no more images.

If you'd like to skip the manual annotation step, we totally understand.  In this case, simply skip this cell and run the next cell instead, where we've prepopulated the groundtruth with pre-annotated bounding boxes.



In [ ]:
gt_boxes = []
annotate(train_images_np, box_storage_pointer=gt_boxes)

# In case you didn't want to label...

Run this cell only if you didn't annotate anything above and
would prefer to just use our preannotated boxes.  Don't forget
to uncomment.

In [ ]:
# gt_boxes = [
#             np.array([[0.436, 0.591, 0.629, 0.712]], dtype=np.float32),
#             np.array([[0.539, 0.583, 0.73, 0.71]], dtype=np.float32),
#             np.array([[0.464, 0.414, 0.626, 0.548]], dtype=np.float32),
#             np.array([[0.313, 0.308, 0.648, 0.526]], dtype=np.float32),
#             np.array([[0.256, 0.444, 0.484, 0.629]], dtype=np.float32)
# ]

# Prepare data for training

Below we add the class annotations (for simplicity, we assume a single class in this colab; though it should be straightforward to extend this to handle multiple classes).  We also convert everything to the format that the training
loop below expects (e.g., everything converted to tensors, classes converted to one-hot representations, etc.).

In [ ]:

# By convention, our non-background classes start counting at 1.  Given
# that we will be predicting just one class, we will therefore assign it a
# `class id` of 1.
duck_class_id = 1
num_classes = 1

category_index = {duck_class_id: {'id': duck_class_id, 'name': 'rubber_ducky'}}

# Convert class labels to one-hot; convert everything to tensors.
# The `label_id_offset` here shifts all classes by a certain number of indices;
# we do this here so that the model receives one-hot labels where non-background
# classes start counting at the zeroth index.  This is ordinarily just handled
# automatically in our training binaries, but we need to reproduce it here.
label_id_offset = 1
train_image_tensors = []
gt_classes_one_hot_tensors = []
gt_box_tensors = []
for (train_image_np, gt_box_np) in zip(
    train_images_np, gt_boxes):
  train_image_tensors.append(tf.expand_dims(tf.convert_to_tensor(
      train_image_np, dtype=tf.float32), axis=0))
  gt_box_tensors.append(tf.convert_to_tensor(gt_box_np, dtype=tf.float32))
  zero_indexed_groundtruth_classes = tf.convert_to_tensor(
      np.ones(shape=[gt_box_np.shape[0]], dtype=np.int32) - label_id_offset)
  gt_classes_one_hot_tensors.append(tf.one_hot(
      zero_indexed_groundtruth_classes, num_classes))
print('Done prepping data.')


# Let's just visualize the rubber duckies as a sanity check


In [ ]:
dummy_scores = np.array([1.0], dtype=np.float32)  # give boxes a score of 100%

plt.figure(figsize=(30, 15))
for idx in range(5):
  plt.subplot(2,3, idx+1)
  plot_detections(
      train_images_np[idx],
      gt_boxes[idx],
      np.ones(shape=[gt_boxes[idx].shape[0]], dtype=np.int32),
      dummy_scores, category_index)
plt.show()

In [ ]:
# Download the checkpoint/ and put it into models/research/object_detection/test_data/

# Create model and restore weights for all but last layer

In this cell we build a single stage detection architecture (RetinaNet) and restore all but the classification layer at the top (which will be automatically randomly initialized).

For simplicity, we have hardcoded a number of things in this colab for the specific RetinaNet architecture at hand (including assuming that the image size will always be 640x640), however it is not difficult to generalize to other model configurations.

In [ ]:
tf.keras.backend.clear_session()

print('Building model and restoring weights for fine-tuning...', flush=True)
num_classes = 1
pipeline_config = 'models/research/object_detection/configs/tf2/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.config'
checkpoint_path = 'models/research/object_detection/test_data/checkpoint/ckpt-26'

# Load pipeline config and build a detection model.
#
# Since we are working off of a COCO architecture which predicts 90
# class slots by default, we override the `num_classes` field here to be just
# one (for our new rubber ducky class).
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
model_config.ssd.num_classes = num_classes
model_config.ssd.freeze_batchnorm = True
detection_model = model_builder.build(
      model_config=model_config, is_training=True)

# Set up object-based checkpoint restore --- RetinaNet has two prediction
# `heads` --- one for classification, the other for box regression.  We will
# restore the box regression head but initialize the classification head
# from scratch (we show the omission below by commenting out the line that
# we would add if we wanted to restore both heads)
fake_box_predictor = tf.compat.v2.train.Checkpoint(
    _base_tower_layers_for_heads=detection_model._box_predictor._base_tower_layers_for_heads,
    # _prediction_heads=detection_model._box_predictor._prediction_heads,
    #    (i.e., the classification head that we *will not* restore)
    _box_prediction_head=detection_model._box_predictor._box_prediction_head,
    )
fake_model = tf.compat.v2.train.Checkpoint(
          _feature_extractor=detection_model._feature_extractor,
          _box_predictor=fake_box_predictor)
ckpt = tf.compat.v2.train.Checkpoint(model=fake_model)
ckpt.restore(checkpoint_path).expect_partial()

# Run model through a dummy image so that variables are created
image, shapes = detection_model.preprocess(tf.zeros([1, 640, 640, 3]))
prediction_dict = detection_model.predict(image, shapes)
_ = detection_model.postprocess(prediction_dict, shapes)
print('Weights restored!')

# Eager mode custom training loop



In [ ]:
tf.keras.backend.set_learning_phase(True)

# These parameters can be tuned; since our training set has 5 images
# it doesn't make sense to have a much larger batch size, though we could
# fit more examples in memory if we wanted to.
batch_size = 4
learning_rate = 0.01
num_batches = 100

# Select variables in top layers to fine-tune.
trainable_variables = detection_model.trainable_variables
to_fine_tune = []
prefixes_to_train = [
  'WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalBoxHead',
  'WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalClassHead']
for var in trainable_variables:
  if any([var.name.startswith(prefix) for prefix in prefixes_to_train]):
    to_fine_tune.append(var)

# Set up forward + backward pass for a single train step.
def get_model_train_step_function(model, optimizer, vars_to_fine_tune):
  """Get a tf.function for training step."""

  # Use tf.function for a bit of speed.
  # Comment out the tf.function decorator if you want the inside of the
  # function to run eagerly.
  @tf.function
  def train_step_fn(image_tensors,
                    groundtruth_boxes_list,
                    groundtruth_classes_list):
    """A single training iteration.

    Args:
      image_tensors: A list of [1, height, width, 3] Tensor of type tf.float32.
        Note that the height and width can vary across images, as they are
        reshaped within this function to be 640x640.
      groundtruth_boxes_list: A list of Tensors of shape [N_i, 4] with type
        tf.float32 representing groundtruth boxes for each image in the batch.
      groundtruth_classes_list: A list of Tensors of shape [N_i, num_classes]
        with type tf.float32 representing groundtruth boxes for each image in
        the batch.

    Returns:
      A scalar tensor representing the total loss for the input batch.
    """
    shapes = tf.constant(batch_size * [[640, 640, 3]], dtype=tf.int32)
    model.provide_groundtruth(
        groundtruth_boxes_list=groundtruth_boxes_list,
        groundtruth_classes_list=groundtruth_classes_list)
    with tf.GradientTape() as tape:
      preprocessed_images = tf.concat(
          [detection_model.preprocess(image_tensor)[0]
           for image_tensor in image_tensors], axis=0)
      prediction_dict = model.predict(preprocessed_images, shapes)
      losses_dict = model.loss(prediction_dict, shapes)
      total_loss = losses_dict['Loss/localization_loss'] + losses_dict['Loss/classification_loss']
      gradients = tape.gradient(total_loss, vars_to_fine_tune)
      optimizer.apply_gradients(zip(gradients, vars_to_fine_tune))
    return total_loss

  return train_step_fn

optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
train_step_fn = get_model_train_step_function(
    detection_model, optimizer, to_fine_tune)

print('Start fine-tuning!', flush=True)
for idx in range(num_batches):
  # Grab keys for a random subset of examples
  all_keys = list(range(len(train_images_np)))
  random.shuffle(all_keys)
  example_keys = all_keys[:batch_size]

  # Note that we do not do data augmentation in this demo.  If you want a
  # a fun exercise, we recommend experimenting with random horizontal flipping
  # and random cropping :)
  gt_boxes_list = [gt_box_tensors[key] for key in example_keys]
  gt_classes_list = [gt_classes_one_hot_tensors[key] for key in example_keys]
  image_tensors = [train_image_tensors[key] for key in example_keys]

  # Training step (forward pass + backwards pass)
  total_loss = train_step_fn(image_tensors, gt_boxes_list, gt_classes_list)

  if idx % 10 == 0:
    print('batch ' + str(idx) + ' of ' + str(num_batches)
    + ', loss=' +  str(total_loss.numpy()), flush=True)

print('Done fine-tuning!')

# Load test images and run inference with new model!

In [ ]:
test_image_dir = 'models/research/object_detection/test_images/ducky/test/'
test_images_np = []
for i in range(1, 50):
  image_path = os.path.join(test_image_dir, 'out' + str(i) + '.jpg')
  test_images_np.append(np.expand_dims(
      load_image_into_numpy_array(image_path), axis=0))

# Again, uncomment this decorator if you want to run inference eagerly
@tf.function
def detect(input_tensor):
  """Run detection on an input image.

  Args:
    input_tensor: A [1, height, width, 3] Tensor of type tf.float32.
      Note that height and width can be anything since the image will be
      immediately resized according to the needs of the model within this
      function.

  Returns:
    A dict containing 3 Tensors (`detection_boxes`, `detection_classes`,
      and `detection_scores`).
  """
  preprocessed_image, shapes = detection_model.preprocess(input_tensor)
  prediction_dict = detection_model.predict(preprocessed_image, shapes)
  return detection_model.postprocess(prediction_dict, shapes)

# Note that the first frame will trigger tracing of the tf.function, which will
# take some time, after which inference should be fast.

label_id_offset = 1
for i in range(len(test_images_np)):
  input_tensor = tf.convert_to_tensor(test_images_np[i], dtype=tf.float32)
  detections = detect(input_tensor)

  plot_detections(
      test_images_np[i][0],
      detections['detection_boxes'][0].numpy(),
      detections['detection_classes'][0].numpy().astype(np.uint32)
      + label_id_offset,
      detections['detection_scores'][0].numpy(),
      category_index, figsize=(15, 20), image_name="gif_frame_" + ('%02d' % i) + ".jpg")

In [ ]:
import IPython
from IPython import display
import imageio
import glob

imageio.plugins.freeimage.download()

anim_file = 'duckies_test.gif'

filenames = glob.glob('gif_frame_*.jpg')
filenames = sorted(filenames)
last = -1
images = []
for i,filename in enumerate(filenames):
  image = imageio.imread(filename)
  images.append(image)

imageio.mimsave(anim_file, images, 'GIF-FI', fps=5)

display.display(display.Image(open(anim_file, 'rb').read()))